In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import geopandas as gpd
from rasterstats import zonal_stats

import warnings
warnings.filterwarnings("ignore")

**Combine all output files generated using droughtPropagationAnalaysis.ipynb differentiatble by lsu and cha identifier**

In [ ]:
# Define the path to  CSV files 
file_path = r"*fullDatabase.csv"

csv_files = glob.glob(file_path)

# Initialize an empty list to hold DataFrames
data_frames = []

# Loop through each file
for file in csv_files:
    # Extract the LSU name from the filename
    lsu_name = os.path.basename(file).split('_')[0]  
    # Read each CSV file
    df = pd.read_csv(file)
    # Add the LSU name as a new column
    df['LSU'] = lsu_name
    # Append the DataFrame to the list
    data_frames.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(data_frames, ignore_index=True)

# Save the combined DataFrame (optional)
combined_df.to_csv(r".\CombinedDroughtDB.csv", index=False)

# Display the combined DataFrame
combined_df

,start_date_precip,end_date_precip,sum_y_values_precip,num_days_precip,start_month_precip,end_month_precip,start_year_precip,end_year_precip,Minor_precip,dbt_pp_precip,...,overlap_end,overlap_duration_days,temporal_overlap_ratio,days_since_last_deficit,precip_deficit_intensity,streamflow_deficit_intensity,precip_deficit_duration_category,streamflow_drought_duration_category,deficit_volume_ratio,LSU
0,1983-06-23,1983-07-14,69.7140,22,6,7,1983,1983,False,22,...,1983-06-26,4,0.181818,NaN,3.168818,0.073322,Short,Long,0.107279,rtu132
1,1984-07-06,1984-07-10,8.6848,5,7,7,1984,1984,False,5,...,1984-07-10,5,1.000000,358.0,1.736960,0.236407,Short,Long,8.547325,rtu132
2,1984-08-24,1984-09-02,12.5428,10,8,8,1984,1984,False,10,...,1984-09-02,2,0.200000,45.0,1.254280,0.255732,Short,Long,6.035075,rtu132
3,1984-09-14,1984-09-25,12.8784,12,9,9,1984,1984,False,12,...,1984-09-25,12,1.000000,12.0,1.073200,0.255732,Short,Long,5.877806,rtu132
4,1985-05-27,1985-06-20,18.9014,25,5,6,1985,1985,False,25,...,1985-06-20,25,1.000000,244.0,0.756056,0.255732,Short,Long,4.004822,rtu132
5,1985-07-14,1985-07-19,6.5298,6,7,7,1985,1985,False,6,...,1985-07-19,6,1.000000,24.0,1.088300,0.485259,Short,Medium,2.898266,rtu132
6,1985-08-31,1985-09-05,5.0634,6,8,9,1985,1985,False,6,...,1985-09-05,0,0.000000,43.0,0.843900,0.363288,Short,Long,12.340618,rtu132
7,1985-09-11,1985-09-15,3.5274,5,9,9,1985,1985,False,5,...,1985-09-15,5,1.000000,6.0,0.705480,0.363288,Short,Long,17.714318,rtu132
8,1986-07-09,1986-07-23,25.7982,15,7,7,1986,1986,False,15,...,1986-07-23,15,1.000000,297.0,1.719880,0.322933,Short,Medium,0.638401,rtu132
9,1986-08-03,1986-08-07,7.0734,5,8,8,1986,1986,False,5,...,1986-08-07,5,1.000000,11.0,1.414680,0.322933,Short,Medium,2.328385,rtu132


In [9]:
# Filter for initiating precipitation deficit events
initiating_df = combined_df[combined_df['relationship_type'] == 'initiating'].copy()
print(initiating_df)
# Convert dates to datetime if they're not already
date_columns = ['start_date_precip', 'end_date_precip', 'start_date_streamflow', 
                'end_date_streamflow', 'overlap_start', 'overlap_end']
for col in date_columns:
    if col in initiating_df.columns:
        if initiating_df[col].dtype == 'object':
            initiating_df[col] = pd.to_datetime(initiating_df[col])

initiating_df['channel_id'] = initiating_df['channel_id'] 

# Calculate derived metrics for drought propagation mechanisms
# 1. Lag: Time between precipitation deficit onset and streamflow drought onset (days)
initiating_df['lag_days'] = (initiating_df['start_date_streamflow'] - 
                             initiating_df['start_date_precip']).dt.days

# 2. Lengthening: Time extension of streamflow drought after precipitation deficit ends (days)
initiating_df['lengthening_days'] = (initiating_df['end_date_streamflow'] - 
                                    initiating_df['end_date_precip']).dt.days

# 3. Attenuation: Ratio of streamflow deficit intensity to precipitation deficit intensity
initiating_df['attenuation_ratio'] = 1- (initiating_df['streamflow_deficit_intensity'] / 
                                     initiating_df['precip_deficit_intensity'])

# Add seasonal information using cyclical encoding
initiating_df['precip_month_sin'] = np.sin(2 * np.pi * initiating_df['precip_deficit_month'] / 12)
initiating_df['precip_month_cos'] = np.cos(2 * np.pi * initiating_df['precip_deficit_month'] / 12)

# Handle missing values
numerical_columns = ['lag_days', 'lengthening_days', 'attenuation_ratio', 
                     'precip_deficit_duration_days', 'precip_deficit_intensity',
                     'streamflow_drought_duration_days', 'streamflow_deficit_intensity']
for col in numerical_columns:
    if initiating_df[col].isnull().any():
        median_val = initiating_df[col].median()
        initiating_df[col] = initiating_df[col].fillna(median_val)
        print(f"Filled {initiating_df[col].isnull().sum()} missing values in {col} with median ({median_val})")



initiating_df.to_csv('processed_initiating_events.csv', index=False)# Save processed data
initiating_df['lsu_id'] = initiating_df['lsu_id'].apply(lambda x: x.replace('rtu', '').strip())

   start_date_precip end_date_precip  sum_y_values_precip  num_days_precip  \
2         1984-08-24      1984-09-02              12.5428               10   
6         1985-08-31      1985-09-05               5.0634                6   
10        1986-08-13      1986-08-18              29.0046                6   
12        1992-07-13      1992-07-18               5.9648                6   
15        1997-08-31      1997-09-12              43.3090               13   
16        2002-08-08      2002-09-10              22.9682               34   
17        2002-09-16      2002-09-25               9.6784               10   
18        2012-08-21      2012-08-25               8.3322                5   
19        2014-07-14      2014-07-20              13.1260                7   
23        2015-06-07      2015-06-22               8.2186               16   

    start_month_precip  end_month_precip  start_year_precip  end_year_precip  \
2                    8                 8               1984  

**Get catchment attributes from SWAT+ lsu/cha shapefile and rater and align with corresponding lsu**

In [ ]:
def extract_dominant_category(lsu_gdf, raster_path, category_names=None, default_value="Unknown"):
    """
    Extract the dominant raster category (land use or soil type) for each LSU polygon.
    
    Parameters:
    -----------
    lsu_gdf : GeoDataFrame
        LSU shapefile data
    raster_path : str
        Path to the raster file (land use or soil)
    category_names : dict
        Mapping of raster values to category names
    default_value : str
        Default value when no data is found
    
    Returns:
    --------
    pd.Series : Dominant category for each LSU
    """
    lsu_gdf = lsu_gdf.copy()
    lsu_gdf['geometry'] = lsu_gdf.geometry.make_valid()
    
    dominant_categories = []
    
    for idx, lsu in lsu_gdf.iterrows():
        lsu_id = lsu['LSUID'] if 'LSUID' in lsu_gdf.columns else idx
        geom = lsu.geometry
        
        try:
            # Get raster statistics for this polygon
            stat = zonal_stats([geom], raster_path, categorical=True)[0]
            
            if stat and len(stat) > 0:
                # Find the most frequent raster value
                dominant_value = max(stat.items(), key=lambda x: x[1])[0]
                
                # Map to category name if mapping provided
                if category_names and dominant_value in category_names:
                    dominant_categories.append(category_names[dominant_value])
                else:
                    dominant_categories.append(str(dominant_value))
            else:
                print(f"No raster values found for LSU {lsu_id}")
                dominant_categories.append(default_value)
                
        except Exception as e:
            print(f"Error extracting raster stats for LSU {lsu_id}: {e}")
            dominant_categories.append(default_value)
    
    return pd.Series(dominant_categories, index=lsu_gdf.index)


def merge_lsu_stream_data(lsu_gdf, stream_gdf):
    """
    Merge LSU data with corresponding stream network attributes using Channel ID.
    
    Parameters:
    -----------
    lsu_gdf : GeoDataFrame
        LSU shapefile data
    stream_gdf : GeoDataFrame
        Stream network shapefile data
    
    Returns:
    --------
    GeoDataFrame : Merged data with stream attributes added to LSUs
    """
    lsu_gdf_copy = lsu_gdf.copy()
    stream_gdf_copy = stream_gdf.copy()
    
    # Verify Channel column exists
    if 'Channel' not in lsu_gdf_copy.columns or 'Channel' not in stream_gdf_copy.columns:
        raise ValueError("Both LSU and Stream dataframes must contain a 'Channel' column")
    
    # Ensure Channel columns are strings for proper matching
    lsu_gdf_copy['Channel'] = lsu_gdf_copy['Channel'].astype(str)
    stream_gdf_copy['Channel'] = stream_gdf_copy['Channel'].astype(str)
    
    # Select stream attributes to merge
    stream_attributes = stream_gdf_copy[[
        'Channel', 'AreaC', 'strmOrder', 'Len2', 'Slo2', 
        'Wid2', 'Dep2', 'MinEl', 'MaxEl'
    ]]
    
    # Optional: Rename columns for clarity
    stream_attributes = stream_attributes.rename(columns={
        'AreaC': 'StreamArea_km2',
        'strmOrder': 'StreamOrder',
        'Len2': 'StreamLength_km',
        'Slo2': 'StreamSlope_fraction',
        'Wid2': 'StreamWidth_m',
        'Dep2': 'StreamDepth_m',
        'MinEl': 'StreamMinElevation_m',
        'MaxEl': 'StreamMaxElevation_m'
    })
    
    # Merge on Channel ID
    merged_df = pd.merge(
        lsu_gdf_copy,
        stream_attributes,
        on='Channel',
        how='left'
    )
    
    return merged_df


def calculate_time_of_concentration_kirpich(merged_df):
    """
    Calculate time of concentration using Kirpich (1940) formula.
    
    Formula: Tc = 0.06648 * (L^0.77) * (S^-0.385)
    Where:
        Tc = time of concentration (hours)
        L = channel length (km)
        S = channel slope (m/m)
    
    Parameters:
    -----------
    merged_df : GeoDataFrame
        LSU data merged with stream attributes
    
    Returns:
    --------
    GeoDataFrame : Input data with added TC_Kirpich_hr column
    """
    df = merged_df.copy()
    
    tc_values = []
    
    for idx, row in df.iterrows():
        try:
            stream_length_km = row['StreamLength_km']
            stream_slope = row['StreamSlope_fraction']
            
            # Calculate only if we have valid data
            if pd.notna(stream_length_km) and pd.notna(stream_slope) and stream_slope > 0:
                # Kirpich formula
                tc_kirpich_hr = 0.06648 * (stream_length_km**0.77) * (stream_slope**(-0.385))
            else:
                tc_kirpich_hr = None
                
        except Exception as e:
            lsu_id = row.get('LSUID', idx)
            print(f"Error calculating time of concentration for LSU {lsu_id}: {e}")
            tc_kirpich_hr = None
        
        tc_values.append(tc_kirpich_hr)
    
    df['TC_Kirpich_hr'] = tc_values
    
    return df


def analyze_lsu_characteristics(lsu_shapefile, landuse_raster, soil_raster, stream_shapefile, 
                              landuse_categories=None, soil_categories=None):
    """
    Extract and compile LSU characteristics for drought analysis.
    
    Parameters:
    -----------
    lsu_shapefile : str
        Path to LSU shapefile
    landuse_raster : str
        Path to land use raster
    soil_raster : str
        Path to soil raster
    stream_shapefile : str
        Path to stream network shapefile
    landuse_categories : dict
        Mapping of land use raster values to category names
    soil_categories : dict
        Mapping of soil raster values to category names
    
    Returns:
    --------
    GeoDataFrame : Complete LSU characteristics table
    """
    # Load shapefiles
    print("Loading LSU shapefile...")
    lsu_gdf = gpd.read_file(lsu_shapefile)
    
    print("Loading stream network shapefile...")
    stream_gdf = gpd.read_file(stream_shapefile)
    
    # Ensure consistent CRS
    if stream_gdf.crs != lsu_gdf.crs:
        print("Aligning coordinate reference systems...")
        stream_gdf = stream_gdf.to_crs(lsu_gdf.crs)
    
    # Extract dominant land use for each LSU
    print("Extracting dominant land use...")
    lsu_gdf['Dominant_LandUse'] = extract_dominant_category(
        lsu_gdf, landuse_raster, landuse_categories
    )
    
    # Extract dominant soil type for each LSU
    print("Extracting dominant soil type...")
    lsu_gdf['Dominant_Soil'] = extract_dominant_category(
        lsu_gdf, soil_raster, soil_categories
    )
    
    # Merge with stream attributes
    print("Merging LSU and stream network data...")
    merged_df = merge_lsu_stream_data(lsu_gdf, stream_gdf)
    
    # Calculate time of concentration
    print("Calculating time of concentration (Kirpich method)...")
    result_df = calculate_time_of_concentration_kirpich(merged_df)
    
    # Create visualizations
    print("Creating visualizations...")
    
    # Land use map
    if 'Dominant_LandUse' in result_df.columns:
        fig, ax = plt.subplots(1, 1, figsize=(12, 10))
        result_df.plot(column='Dominant_LandUse', categorical=True, legend=True, ax=ax)
        stream_gdf.plot(ax=ax, color='blue', linewidth=0.5, alpha=0.7)
        ax.set_title('LSUs by Dominant Land Use with Stream Network')
        ax.axis('off')
        plt.tight_layout()
        plt.savefig('lsu_landuse_map.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # Stream order map
    if 'StreamOrder' in result_df.columns:
        fig, ax = plt.subplots(1, 1, figsize=(12, 10))
        result_df.plot(column='StreamOrder', cmap='viridis', legend=True, ax=ax)
        ax.set_title('LSUs by Stream Order')
        ax.axis('off')
        plt.tight_layout()
        plt.savefig('lsu_stream_order_map.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # Select output columns (only those needed for drought analysis)
    output_columns = [
        'LSUID', 'Subbasin', 'Channel', 'Area', 'Slope', 
        'Lat', 'Lon', 'Elev',
        'Dominant_LandUse', 'Dominant_Soil', 
        'StreamArea_km2', 'StreamOrder', 'StreamLength_km', 'StreamSlope_fraction',
        'StreamWidth_m', 'StreamDepth_m', 'StreamMinElevation_m', 'StreamMaxElevation_m',
        'TC_Kirpich_hr'
    ]
    
    # Keep only existing columns
    output_columns = [col for col in output_columns if col in result_df.columns]
    
    # Save results
    print("Saving results...")
    result_df[output_columns].to_csv('lsu_characteristics.csv', index=False)
    
    # Create summary statistics
    summary_stats = result_df[output_columns].describe()
    summary_stats.to_csv('lsu_characteristics_summary.csv')
    
    print(f"\nAnalysis complete!")
    print(f"Total LSUs processed: {len(result_df)}")
    print(f"Results saved to: lsu_characteristics.csv")
    print(f"Summary statistics saved to: lsu_characteristics_summary.csv")
    
    return result_df


# Define file paths
lsu_shapefile = "exampleData\lsu121\lsu121.shp"
soil_raster = "exampleData\soil.tif"
landuse_raster = "exampleData\landuse.tif"
stream_shapefile = 'exampleData\rivsLSU121\rivs1.shp'

# Land use categories mapping
landuse_categories = {
    20: 'RNGB',
    30: 'RNGE',
    41: 'AGRR',
	50: 'URHD',
#.. Add more categories as needed
}

# Soil categories mapping. Define Soil categories based on your raster values
soil_categories = {
    0: 'shu-0',
    1: 'shu-1',
	2: 'shu-2',
	12: 'shu-12',
  #.. Add more categories as needed
}

# Run the analysis
if __name__ == "__main__":
    lsu_characteristics = analyze_lsu_characteristics(
        lsu_shapefile, 
        landuse_raster, 
        soil_raster, 
        stream_shapefile,
        landuse_categories,
        soil_categories
    )

Loading LSU shapefile...
Loading stream network shapefile...
Extracting dominant land use...
Extracting dominant soil type...
Merging LSU and stream network data...
Calculating time of concentration (Kirpich method)...
Creating visualizations...
Saving results...

Analysis complete!
Total LSUs processed: 67
Results saved to: lsu_characteristics.csv
Summary statistics saved to: lsu_characteristics_summary.csv


In [ ]:
lsu_numbers = ['121',
#,...add more as needed
]
# Define the list of LSUs
lsu_ids = [f'rtu{value}' for value in lsu_numbers]
# Load the file into a DataFrame (assumes space-separated values)
lsu_file_path = r'path\to\lsunit_wb_day.txt'
print("Loading data...")
with open(lsu_file_path, 'r') as file:
    lines = file.readlines()

data = []
for line in lines:
    data.append(line.split())

df = pd.DataFrame(data)
df.columns = df.iloc[1]  # Use the second row as column names
df = df[3:]  # Drop unnecessary rows
df.columns = df.columns.str.strip()  # Clean up column names

# Ensure proper data types
df = df.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.name not in ['name'] else col)
df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].astype(str).agg('-'.join, axis=1))

# Process each LSU
for lsu_id in lsu_ids:
    print(f"Processing LSU: {lsu_id}...")
    
    # Filter data for the LSU
    filtered_postp_data = df[df['name'] == lsu_id]
    
    # Rolling calculations
    filtered_postp_data['precip_roll_5d'] = filtered_postp_data['precip'].rolling(window=5).mean()

    filtered_postp_data.reset_index(drop=True, inplace=True)

    # Prepare cumulative DataFrame
    df_cummulative = pd.DataFrame({
        'date': filtered_postp_data['date'],
        'Cumulative_Difference_Precip': filtered_postp_data['precip_roll_5d'],
        'Cumulative_Difference_ET': filtered_postp_data['et'],
        'Cumulative_Difference_SW_300': filtered_postp_data['sw_300']
    }).reset_index(drop=True)

    # Load drought event file for the current LSU
    drought_event_lsu_file = f'path/to/{lsu_id}_fullDatabase.csv'
    drought_event_lsu = pd.read_csv(drought_event_lsu_file)
    drought_event_lsu['start_date_precip'] = pd.to_datetime(drought_event_lsu['start_date_precip'])

    # Define time windows
    time_windows = [7, 10, 20]
    for window in time_windows:
        drought_event_lsu[f'Precip_Avg_{window}d'] = None
        drought_event_lsu[f'ET_Avg_{window}d'] = None
        drought_event_lsu[f'SW_300_Avg_{window}d'] = None

    # Calculate averages
    for index, event in drought_event_lsu.iterrows():
        start_date = event['start_date_precip']
        for window in time_windows:
            window_start_date = start_date - pd.Timedelta(days=window)
            window_data = df_cummulative[
                (df_cummulative['date'] >= window_start_date) &
                (df_cummulative['date'] < start_date)
            ]
            if not window_data.empty:
                avg_precip = window_data['Cumulative_Difference_Precip'].mean()
                avg_et = window_data['Cumulative_Difference_ET'].mean()
                avg_sw_300 = window_data['Cumulative_Difference_SW_300'].mean()
            else:
                avg_precip = None
                avg_et = None
                avg_sw_300 = None

            drought_event_lsu.at[index, f'Precip_Avg_{window}d'] = avg_precip
            drought_event_lsu.at[index, f'ET_Avg_{window}d'] = avg_et
            drought_event_lsu.at[index, f'SW_300_Avg_{window}d'] = avg_sw_300
            

    # Save updated DataFrame to a new CSV file
    output_file = f'./antecedent/Processed_P_ET_SW_300_{lsu_id}.csv'
    drought_event_lsu.to_csv(output_file, index=False)
    print(f"Saved results to {output_file}.")

print("Processing completed for all LSUs.")

rtu_list = [f'rtu{value}' for value in lsu_numbers]

all_rtus = []
for rtu in rtu_list:
    file_path = f'./antecedent/Processed_P_ET_SW_300_{rtu}.csv'  
    rtu_df = pd.read_csv(file_path)
    all_rtus.append(rtu_df)

# Concatenate all DataFrames into one
combined_all_lsu_df = pd.concat(all_rtus, ignore_index=True)

# Check the resulting DataFrame
combined_all_lsu_df


Loading data...
Processing LSU: rtu132...
Saved results to C:/Users/ethio/Documents/Postprocessing/awash_pieces_drought_metrics/modifiedAutomationDroughtMetrics/antecedent/Processed_P_ET_SW_300_rtu132.csv.
Processing LSU: rtu131...
Saved results to C:/Users/ethio/Documents/Postprocessing/awash_pieces_drought_metrics/modifiedAutomationDroughtMetrics/antecedent/Processed_P_ET_SW_300_rtu131.csv.
Processing LSU: rtu122...
Saved results to C:/Users/ethio/Documents/Postprocessing/awash_pieces_drought_metrics/modifiedAutomationDroughtMetrics/antecedent/Processed_P_ET_SW_300_rtu122.csv.
Processing LSU: rtu121...
Saved results to C:/Users/ethio/Documents/Postprocessing/awash_pieces_drought_metrics/modifiedAutomationDroughtMetrics/antecedent/Processed_P_ET_SW_300_rtu121.csv.
Processing LSU: rtu142...
Saved results to C:/Users/ethio/Documents/Postprocessing/awash_pieces_drought_metrics/modifiedAutomationDroughtMetrics/antecedent/Processed_P_ET_SW_300_rtu142.csv.
Processing LSU: rtu152...
Saved re

,level_0_precip,index_precip,start_date_precip,end_date_precip,sum_y_values_precip,num_days_precip,start_month_precip,end_month_precip,start_year_precip,end_year_precip,...,deficit_volume_ratio,Precip_Avg_7d,ET_Avg_7d,SW_300_Avg_7d,Precip_Avg_10d,ET_Avg_10d,SW_300_Avg_10d,Precip_Avg_20d,ET_Avg_20d,SW_300_Avg_20d
0,2.0,4.0,1983-06-23,1983-07-14,69.6670,21,6,7,1983,1983,...,0.106554,9.260714,2.133286,46.792429,7.74716,2.1338,46.1520,5.40614,1.99560,44.3185
1,12.0,24.0,1984-08-24,1984-09-02,11.8040,4,8,9,1984,1984,...,6.411189,7.656371,2.697143,49.308857,7.38780,2.6390,49.2325,7.71131,2.51970,50.1901
2,13.0,26.0,1984-09-16,1984-09-25,9.5502,9,9,9,1984,1984,...,7.924198,9.234943,2.348714,45.449000,11.22416,2.3820,47.3422,8.68640,2.56445,48.2526
3,16.0,32.0,1985-05-27,1985-06-18,17.9780,22,5,6,1985,1985,...,4.209460,1.190143,1.884286,36.064429,0.92908,1.7979,36.3046,2.85917,1.82230,39.9765
4,28.0,51.0,1986-07-09,1986-07-23,20.7884,8,7,7,1986,1986,...,0.790951,7.222314,1.968571,49.261571,7.36508,1.8829,48.2373,5.84821,1.68090,47.5800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,236.0,376.0,2014-06-25,2014-07-07,14.7336,12,6,7,2014,2014,...,1.260028,0.635000,1.155000,20.452429,0.58972,1.2217,21.1331,1.02375,1.47965,23.5694
1226,238.0,378.0,2014-07-25,2014-08-05,29.4078,11,7,8,2014,2014,...,0.631287,6.461000,1.934857,38.058286,4.80500,1.8085,35.5248,4.53220,1.80890,32.1210
1227,240.0,380.0,2014-08-12,2014-08-20,7.3594,4,8,8,2014,2014,...,2.522591,5.330686,2.257714,39.707857,4.49158,2.2615,38.4200,3.51059,1.91185,36.1065
1228,249.0,391.0,2015-08-25,2015-08-31,5.7818,6,8,8,2015,2015,...,4.247008,8.661971,3.046714,41.438000,7.19934,2.9024,41.1185,5.26578,2.81635,40.3995


In [19]:
lsu_characteristics['LSUID'] = lsu_characteristics['LSUID'].astype(str).str.zfill(3)
combined_all_lsu_df['lsu_id'] = combined_all_lsu_df['lsu_id'].apply(lambda x: x.replace('rtu', '').strip())
merged_df_form_antecendent = combined_all_lsu_df.merge(lsu_characteristics, left_on='lsu_id', right_on='LSUID', how='inner')

# Check the shape of the merged dataframe
print(f"Original initiating_df shape: {combined_all_lsu_df.shape}")
print(f"Original lsu_characteristics shape: {lsu_characteristics.shape}")
print(f"Merged dataframe shape: {merged_df_form_antecendent.shape}")

# Preview the merged dataframe
merged_df_form_antecendent.head()

Original initiating_df shape: (1230, 71)
Original lsu_characteristics shape: (67, 27)
Merged dataframe shape: (1230, 98)


,level_0_precip,index_precip,start_date_precip,end_date_precip,sum_y_values_precip,num_days_precip,start_month_precip,end_month_precip,start_year_precip,end_year_precip,...,Dominant_Soil,StreamArea_km2,StreamOrder,StreamLength_km,StreamSlope_fraction,StreamWidth_m,StreamDepth_m,StreamMinElevation_m,StreamMaxElevation_m,TC_Kirpich_hr
0,2.0,4.0,1983-06-23,1983-07-14,69.6670,21,6,7,1983,1983,...,shu-2,76550.0,1,28056.3,0.584539,69.335689,1.85149,2060.0,2224.0,217.495982
1,12.0,24.0,1984-08-24,1984-09-02,11.8040,4,8,9,1984,1984,...,shu-2,76550.0,1,28056.3,0.584539,69.335689,1.85149,2060.0,2224.0,217.495982
2,13.0,26.0,1984-09-16,1984-09-25,9.5502,9,9,9,1984,1984,...,shu-2,76550.0,1,28056.3,0.584539,69.335689,1.85149,2060.0,2224.0,217.495982
3,16.0,32.0,1985-05-27,1985-06-18,17.9780,22,5,6,1985,1985,...,shu-2,76550.0,1,28056.3,0.584539,69.335689,1.85149,2060.0,2224.0,217.495982
4,28.0,51.0,1986-07-09,1986-07-23,20.7884,8,7,7,1986,1986,...,shu-2,76550.0,1,28056.3,0.584539,69.335689,1.85149,2060.0,2224.0,217.495982


In [ ]:
# Initialize a new dataframe for aggregated event data
aggregated_data = []

# Create separate dataframes for different event types
initiating_events = merged_df_form_antecendent[merged_df_form_antecendent['relationship_type'] == 'initiating']
sustaining_events = merged_df_form_antecendent[merged_df_form_antecendent['relationship_type'] == 'sustaining']

# Create a list of unique event IDs
#unique_events = merged_df_form_antecendent['event_id'].unique()
unique_events = merged_df_form_antecendent[['event_id', 'lsu_id']].drop_duplicates().values

unique_events

for event_combo in unique_events:
    event_id, lsu_id = event_combo
    
    # Get all records for this event and lsu combination
    event_records = merged_df_form_antecendent[
        (merged_df_form_antecendent['event_id'] == event_id) & 
        (merged_df_form_antecendent['lsu_id'] == lsu_id)
    ]
    
    # Check if there's an initiating event
    init_events = event_records[event_records['relationship_type'] == 'initiating']
    sust_events = event_records[event_records['relationship_type'] == 'sustaining']
    
    # Base data from initiating event (if it exists, otherwise from first sustaining event)
    if len(init_events) > 0:
        base_data = init_events.iloc[0].to_dict()
        base_data['has_initiating'] = 1
        base_data['init_duration'] = init_events.iloc[0]['precip_deficit_duration_days']
        base_data['init_intensity'] = init_events.iloc[0]['precip_deficit_intensity']
        base_data['init_overlap_duration'] = init_events.iloc[0]['overlap_duration_days']
        base_data['init_temporal_overlap'] = init_events.iloc[0]['temporal_overlap_ratio']
        base_data['init_stress'] = base_data['init_duration'] * base_data['init_intensity']
    else:
        if len(sust_events) > 0:
            base_data = sust_events.iloc[0].to_dict()
        else:
            # Handle the case where there are no events at all
            continue
        base_data['has_initiating'] = 0
        base_data['init_duration'] = 0
        base_data['init_intensity'] = 0
        base_data['init_overlap_duration'] = 0
        base_data['init_temporal_overlap'] = 0
        base_data['init_stress'] = 0
    
    # Initialize sustaining event columns (up to 6 events, adjust as needed))
    for i in range(1, 7):
        base_data[f'sust_duration_{i}'] = 0
        base_data[f'sust_intensity_{i}'] = 0
        base_data[f'sust_overlap_duration_{i}'] = 0
        base_data[f'sust_temporal_overlap_{i}'] = 0
        base_data[f'sust_stress_{i}'] = 0
        base_data[f'sust_start_date_{i}'] = None
        base_data[f'sust_end_date_{i}'] = None
    
    # Populate data for each sustaining event (up to 6)
    if len(sust_events) > 0:
        for i, event in enumerate(sust_events.iterrows()):
            if i >= 6:  # Limit to 6 sustaining events
                break
                
            # Get the event data
            _, event_data = event
            
            # Fill in the data for this sustaining event
            event_idx = i + 1  # 1-indexed for column names
            base_data[f'sust_duration_{event_idx}'] = event_data['precip_deficit_duration_days']
            base_data[f'sust_intensity_{event_idx}'] = event_data['precip_deficit_intensity']
            base_data[f'sust_overlap_duration_{event_idx}'] = event_data['overlap_duration_days']
            base_data[f'sust_temporal_overlap_{event_idx}'] = event_data['temporal_overlap_ratio']
            base_data[f'sust_stress_{event_idx}'] = event_data['precip_deficit_duration_days'] * event_data['precip_deficit_intensity']
            
            # Add dates if they exist
            if 'start_date' in event_data:
                base_data[f'sust_start_date_{event_idx}'] = event_data['start_date']
            if 'end_date' in event_data:
                base_data[f'sust_end_date_{event_idx}'] = event_data['end_date']
        
        # Still calculate aggregated metrics for convenience
        base_data['total_sust_duration'] = sust_events['precip_deficit_duration_days'].sum()
        base_data['avg_sust_intensity'] = sust_events['precip_deficit_intensity'].mean()
        base_data['max_sust_intensity'] = sust_events['precip_deficit_intensity'].max()
        base_data['num_sustaining_events'] = len(sust_events)
        base_data['total_overlap_duration'] = sust_events['overlap_duration_days'].sum()
        
        # Weighted average temporal overlap ratio
        weights = sust_events['precip_deficit_duration_days']
        base_data['weighted_overlap_ratio'] = (
            (sust_events['temporal_overlap_ratio'] * weights).sum() / weights.sum()
        )
        
        # Calculate total sustaining stress
        base_data['sust_stress'] = sust_events['precip_deficit_duration_days'].multiply(
            sust_events['precip_deficit_intensity']).sum()
    else:
        # No sustaining events - zeros are already initialized
        base_data['total_sust_duration'] = 0
        base_data['avg_sust_intensity'] = 0
        base_data['max_sust_intensity'] = 0
        base_data['num_sustaining_events'] = 0
        base_data['total_overlap_duration'] = 0
        base_data['weighted_overlap_ratio'] = 0
        base_data['sust_stress'] = 0
    
    # Create combined/interaction features
    if base_data['has_initiating'] == 1 and base_data['num_sustaining_events'] > 0:
        base_data['sust_init_duration_ratio'] = base_data['total_sust_duration'] / base_data['init_duration']
        base_data['sust_init_intensity_ratio'] = base_data['avg_sust_intensity'] / base_data['init_intensity']
        base_data['total_stress'] = base_data['init_stress'] + base_data['sust_stress']
        base_data['sust_stress_fraction'] = base_data['sust_stress'] / base_data['total_stress']
    else:
        base_data['sust_init_duration_ratio'] = 0
        base_data['sust_init_intensity_ratio'] = 0
        base_data['total_stress'] = base_data['init_stress']
        base_data['sust_stress_fraction'] = 0
    
    aggregated_data.append(base_data)

# Create dataframe from aggregated data
agg_df = pd.DataFrame(aggregated_data)
agg_df.to_csv('aggregated_drought_events_and_descriptors.csv', index=False)

,level_0_precip,index_precip,start_date_precip,end_date_precip,sum_y_values_precip,num_days_precip,start_month_precip,end_month_precip,start_year_precip,end_year_precip,...,avg_sust_intensity,max_sust_intensity,num_sustaining_events,total_overlap_duration,weighted_overlap_ratio,sust_stress,sust_init_duration_ratio,sust_init_intensity_ratio,total_stress,sust_stress_fraction
0,2.0,4.0,1983-06-23,1983-07-14,69.6670,21,6,7,1983,1983,...,3.166682,3.166682,1,4,0.181818,69.6670,0.000000,0.000000,0.0000,0.000000
1,12.0,24.0,1984-08-24,1984-09-02,11.8040,4,8,9,1984,1984,...,0.868336,0.955020,2,33,1.000000,27.5282,3.300000,0.735629,39.3322,0.699890
2,28.0,51.0,1986-07-09,1986-07-23,20.7884,8,7,7,1986,1986,...,1.385893,1.385893,1,15,1.000000,20.7884,0.000000,0.000000,0.0000,0.000000
3,30.0,53.0,1986-08-13,1986-08-18,27.6914,5,8,8,1986,1986,...,0.000000,0.000000,0,0,0.000000,0.0000,0.000000,0.000000,27.6914,0.000000
4,94.0,148.0,1992-07-13,1992-07-18,5.4502,5,7,7,1992,1992,...,0.000000,0.000000,0,0,0.000000,0.0000,0.000000,0.000000,5.4502,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,24.0,42.0,1986-07-02,1986-07-19,44.5404,17,7,7,1986,1986,...,2.507539,2.917475,3,34,1.000000,84.9256,0.000000,0.000000,0.0000,0.000000
644,33.0,56.0,1987-07-15,1987-07-21,18.2480,6,7,7,1987,1987,...,1.653836,1.653836,1,11,1.000000,18.1922,1.571429,0.634418,36.4402,0.499234
645,236.0,376.0,2014-06-25,2014-07-07,14.7336,12,6,7,2014,2014,...,1.467238,2.450650,3,34,1.000000,51.5008,0.000000,0.000000,0.0000,0.000000
646,249.0,391.0,2015-08-25,2015-08-31,5.7818,6,8,8,2015,2015,...,0.825971,0.825971,1,7,1.000000,5.7818,0.000000,0.000000,0.0000,0.000000
